## Output Parsers

There are many types of Output Parsers which can parse the LLM output and return a specific schema

#### Create an LLM

In [1]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

So if we communicate with this LLM, we will get an object as return

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template = """
        Provided the word form user, give meaning, example and etymology of the word in 1 sentence each. 
        Word: {word}
    """,
    input_variable=['word']
)

chain = prompt | llm

chain.invoke("Llama")

AIMessage(content='Llama is an animal that belongs to the siberian camelids family, commonly known as llamas or mares.\nEtymology:\nThe term "llama" comes from the Latin "laima," which means "sheep." In the traditional nomadic cultures of central Asia and northern Europe, llamas were often kept by women for their milk and wool.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:0.5b', 'created_at': '2025-03-01T11:59:37.64717774Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1790996301, 'load_duration': 27471278, 'prompt_eval_count': 60, 'prompt_eval_duration': 51000000, 'eval_count': 76, 'eval_duration': 1702000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-452254c7-b373-41b8-baab-b646a88d5d44-0', usage_metadata={'input_tokens': 60, 'output_tokens': 76, 'total_tokens': 136})

### Str Output Parser

Now we get lot of information. Now lets say we just need the String Output, we can use StrOutputParser

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template = """
        Provided the word form user, give meaning, example and etymology of the word in 1 sentence each. 
        Word: {word}
    """,
    input_variable=['word']
)

chain = prompt | llm | StrOutputParser()

chain.invoke("Llama")

'Llama is the term for an animal that can only be seen from behind and has a white face, with long legs, and a short body.\n- **Meaning**: An animal that can only be seen from behind.\n- **Example**: "The llama in the photograph is seen from behind while running down the street. "\n- **Etymology**: The word comes from Old Italian "laima," which means "white." In medieval Latin, this term referred to a white horse or any animal with a white face.\n\nLlama\n- **Meaning**: A type of llama.\n- **Example**: "There are many lamas in the area, and some are very old."\n- **Etymology**: The word "lama" comes from Old Spanish "lima," which means "white." In medieval Latin, it referred to a white horse or any animal with a white face.'

### Structured Output Parser

Now lets say we want the output in a specific structure. We can use Structured Output parser to instruct the LLM to return data in specific structure

In [6]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Lets generate the schema in which we need the response
return_schema = [
    ResponseSchema(name="word", description="The word entered by User"),
    ResponseSchema(name="meaning", description="The meaning of word entered by User"),
    ResponseSchema(name="example", description="Example for the word entered by user"),
    ResponseSchema(name="etymology", description="Etymology for the word entered by user"),
]

# Lets create Output Parser
structured_output = StructuredOutputParser.from_response_schemas(return_schema)

# When we create a structured output parser, it adds formatting instructions to the prompt
format_instruct = structured_output.get_format_instructions()
print(format_instruct)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"word": string  // The word entered by User
	"meaning": string  // The meaning of word entered by User
	"example": string  // Example for the word entered by user
	"etymology": string  // Etymology for the word entered by user
}
```


In [9]:
# So now we can re-create the prompt template as follow - 
from langchain_core.prompts import PromptTemplate
from pprint import pprint

prompt = PromptTemplate(
    template = """
        Provided the word form user, give meaning, example and etymology of the word in 1 sentence each. 
        Word: {word}
        ----------------
        {format_instruct}
    """,
    input_variable=['word'],
    partial_variables={"format_instruct": format_instruct}
)

chain = prompt | llm | structured_output

pprint(chain.invoke("Llama"))

{'etymology': "Ancient Latin term meaning 'dog-like' or 'small dog', with "
              "references to its woolly coat, which is why it's also known as "
              'a llama.',
 'example': 'The lama was the most famous companion of Jesus Christ and later '
            'became known as a symbol of loyalty and devotion.',
 'meaning': 'A llama is a small, woolly dog breed that originated in South '
            'America and has been domesticated since ancient times.',
 'word': 'Llama'}
